**Author: S. Grunert**     
**Revised: February 1, 2016**     
**Topic: ETL for NOAA buoy data gathering, clean-up, and storage in sqlite.**     
**Versions: Py 2.7** 


###EXTRACT: Download Buoy Data from NOAA.

In [16]:
#Download files for each buoy-month.

import urllib

mth = raw_input("Enter Month like Dec: ")
buoys = ['46240','46236','46239']
year = ['_2015_']

for buoy in buoys:
    url = 'http://ndbc-load.nws.noaa.gov/data/stdmet/' + mth + "/" + buoy + ".txt"
    uh = urllib.urlopen(url)
    data = uh.read()
    print url
    filename = 'C:\\buoy_data\\' + buoy + year + mth + '.txt'
    outfile = open(filename, "w")
    outfile.write(data)
    outfile.close()


Enter Month like Dec: Dec
http://ndbc-load.nws.noaa.gov/data/stdmet/Dec/46240.txt
http://ndbc-load.nws.noaa.gov/data/stdmet/Dec/46236.txt
http://ndbc-load.nws.noaa.gov/data/stdmet/Dec/46239.txt


###TRANSFORM: Add buoy ID, pick columns, and merge files. 

In [9]:
#Combine individual buoy downloads into a single file picking applicable columns.

import glob
import os
import re

os.chdir('C:\\buoy_data\\data\\')
filelist = glob.glob("*.txt")
print filelist
dhandle = open('import.txt','a')

for file in filelist:
    if len(file) < 12: continue
    buoy = re.findall('^[0-9]+',file)
    #print buoy
    fhandle = open(file)
    try:
        for line in fhandle:
            if line.startswith('#'): continue
            lvalues = line.split()
            if len(lvalues) < 1: continue
            row = list()
            row.append(buoy[0])
            row.append('\t')
            row.append(lvalues[0])
            row.append('\t')
            row.append(lvalues[1])
            row.append('\t')
            row.append(lvalues[2])
            row.append('\t')
            row.append(lvalues[3])
            row.append('\t')
            row.append(lvalues[4])
            row.append('\t')
            row.append(lvalues[8])
            row.append('\t')
            row.append(lvalues[11])
            row.append('\t')
            row.append(lvalues[14])
            row.append('\n')
            output = ''.join(map(str, row))
            dhandle.write(output)
    except:
        print("Error. Check the data.")
    fhandle.close()
dhandle.close()

['46236_2015_Dec.txt', '46236_2015_Nov.txt', '46236_2015_Oct.txt', '46236_2015_Sep.txt', '46239_2015_Dec.txt', '46239_2015_Nov.txt', '46239_2015_Oct.txt', '46239_2015_Sep.txt', '46240_2015_Dec.txt', '46240_2015_Nov.txt', '46240_2015_Oct.txt', '46240_2015_Sep.txt']


###LOAD: Insert data into a sqlite database.

In [11]:
#Insert the buoy data into a sqlite table for convenient SQL manipulation.

import sqlite3
import os
import time

os.chdir('C:\\buoy_data\\')
fhand = open("C:\\buoy_data\\data\\import.txt")
conn = sqlite3.connect('buoy.sqlite')
cur = conn.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS buoy_data
    (buoy TEXT,year TEXT,month TEXT,day TEXT,hour TEXT,minute TEXT,WVHT TEXT,MWD TEXT,WTMP TEXT)''')
count = 0
try:
    for line in fhand :
        #if count >10: break
        line = line.rstrip()
        line = line.split('\t')
        cur.execute('''INSERT INTO buoy_data (buoy,year,month,day,hour,minute,WVHT,MWD,WTMP)
            VALUES(?,?,?,?,?,?,?,?,?)''',(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8]))
        #conn.commit()
        #time.sleep(0.1)
        #print line
        count = count + 1
    conn.commit()
except:
    conn.rollback()
conn.close()
fhand.close()
print "Rows Loaded:" , count

Rows Loaded: 16171


###VISUALIZE: Download a file suitable for Tableau

In [12]:
#Execute a specific SQL statement to prepare buoy data for visualization.

import sqlite3
import os
import re

os.chdir('C:\\buoy_data\\')
conn = sqlite3.connect('buoy.sqlite')
cur = conn.cursor()
cur.execute('''Select
    buoy
    ,year
    ,month
    ,day
    ,round(max(WVHT),2)    as max_WVHT
    ,round(avg(WVHT),2)    as avg_WVHT
    ,round(min(WVHT),2)    as min_WVHT
    ,round(avg(MWD),0)     as avg_MWD
    ,round(max(WTMP),1)    as max_WTMP
    ,round(avg(WTMP),1)    as avg_WTMP
    ,round(min(WTMP),1)    as min_WTMP
From
    buoy_data
Where
    WVHT >= 0 and WVHT <= 20
    and MWD >= 0 and MWD <= 361
    and WTMP >= 0 and WTMP <= 30
Group By
    buoy
    ,year
    ,month
    ,day''')
dhandle = open('agg_data.csv','a')
count = 0
try:
    for row in cur:
        #if count >10: break
        row = str(row)
        row = row.replace('u','')
        row = row.replace('(','')
        row = row.replace(')','')
        row = row.replace('\'','')
        #print row
        count = count + 1
        dhandle.write(row + "\n")
except:
    conn.close()
    dhandle.close()
conn.close()
dhandle.close()
print "Rows to File:" , count


Rows to File: 352
